In [5]:
!pip install pgmpy

  Obtaining dependency information for pgmpy from https://files.pythonhosted.org/packages/c7/e6/e451590c2341b3d59d7b613e1af80daefd9e2873f7c9ad3d498ad84e7f44/pgmpy-0.1.26-py3-none-any.whl.metadata
  Using cached pgmpy-0.1.26-py3-none-any.whl.metadata (9.1 kB)
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/0d/4a/e51420d46cfc90562e85af2fee912237c662ab31140ab179e49bd69401d6/torch-2.5.1-cp311-cp311-win_amd64.whl.metadata
  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/fe/df/e3a1f3f008db8d2b199ded2168014c7784b8027714b74d802c892815fd72/xgboost-2.1.2-py3-none-win_amd64.whl.metadata
  Using cached xgboost-2.1.2-py3-none-win_amd64.whl.metadata (2.1 kB)
  Obtaining dependency information for google-generativeai from https://files.pythonhosted.org/packages/e9/2f/b5c1d62e94409ed98d5425e83b8e6d3dd475b611be272f561b1a545d273a/google_generativeai-0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.17.0 requires protobuf!=4.24.0,<5.0.0,>=3.19.6, but you have protobuf 5.28.3 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.28.3 which is incompatible.


In [6]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Step 1: Define the structure of the Bayesian Network
model = BayesianNetwork([('Battery', 'Car'), ('Fuel', 'Car')])

# Step 2: Define Conditional Probability Distributions (CPDs)

# Battery: Prior probability (no parents)
cpd_battery = TabularCPD(
    variable='Battery', variable_card=2,
    values=[[0.2], [0.8]],  # 20% chance Dead, 80% Not Dead
    state_names={'Battery': ['Dead', 'Not Dead']}
)

# Fuel: Prior probability (no parents)
cpd_fuel = TabularCPD(
    variable='Fuel', variable_card=2,
    values=[[0.3], [0.7]],  # 30% Empty, 70% Not Empty
    state_names={'Fuel': ['Empty', 'Not Empty']}
)

# Car: Conditional probability depending on Battery and Fuel
cpd_car = TabularCPD(
    variable='Car', variable_card=2,
    values=[
        [0.9, 0.8, 0.1, 0.0],  # Probability Car Starts
        [0.1, 0.2, 0.9, 1.0]   # Probability Car Does Not Start
    ],
    evidence=['Battery', 'Fuel'],
    evidence_card=[2, 2],  # Each evidence has two states
    state_names={
        'Car': ['Starts', 'Does Not Start'],
        'Battery': ['Dead', 'Not Dead'],
        'Fuel': ['Empty', 'Not Empty']
    }
)

# Step 3: Add CPDs to the model
model.add_cpds(cpd_battery, cpd_fuel, cpd_car)

# Step 4: Validate the model
assert model.check_model(), "Model is invalid!"

# Step 5: Perform inference
inference = VariableElimination(model)

# Query the probability of the car starting
result = inference.query(variables=['Car'], evidence={'Battery': 'Dead', 'Fuel': 'Empty'})
print(result)

# Query the probability of the car starting given the battery is not dead
result = inference.query(variables=['Car'], evidence={'Battery': 'Not Dead', 'Fuel': 'Not Empty'})
print(result)


INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


+---------------------+------------+
| Car                 |   phi(Car) |
+=====================+============+
| Car(Starts)         |     0.9000 |
+---------------------+------------+
| Car(Does Not Start) |     0.1000 |
+---------------------+------------+
+---------------------+------------+
| Car                 |   phi(Car) |
+=====================+============+
| Car(Starts)         |     0.0000 |
+---------------------+------------+
| Car(Does Not Start) |     1.0000 |
+---------------------+------------+
